# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [89]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct,count,desc,udf,avg, round
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import Window
from pyspark.sql.types import IntegerType
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [3]:
spark=SparkSession \
    .builder \
    .getOrCreate()

In [4]:
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1593740569233'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'f3e0d7e67d14'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '44013'),
 ('spark.app.name', 'pyspark-shell')]

In [5]:
path= "data/sparkify_log_small.json"
user_log=spark.read.json(path)

In [6]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [7]:
user_log.select("page").dropDuplicates().show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [8]:
# TODO: write your code to answer question 1
user_log.select("page").where(user_log['userId']=="").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [55]:
# filter for users with blank id
blank=user_log.where(user_log.userId=='').select('page').dropDuplicates()

# list of all possible pages
allpages=user_log.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
set(allpages.collect())-set(blank.collect())

{Row(page='Downgrade'),
 Row(page='Error'),
 Row(page='Logout'),
 Row(page='NextSong'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='Submit Downgrade'),
 Row(page='Submit Upgrade'),
 Row(page='Upgrade')}

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [9]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select("*").where(user_log['userId']=="").show()

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|            0|    null|  null| free|    null|   PUT|Login|        null|     5598|null|   307|1513721196284|     null|      |
|  null|Logged Out|     null|  null|           26|    null|  null| paid|    null|   GET| Home|        null|      428|null|   200|1513721274284|     null|      |
|  null|Logged Out|     null|  null|            5|    null|  null| free|    null|   GET| Home|        null|     2941|null|   200|1513722009284|     null|      |
|  null|Logged Out|     null|  nul

# Question 3

How many female users do we have in the data set?

In [10]:
user_log.select('gender').dropDuplicates().show()

+------+
|gender|
+------+
|     F|
|  null|
|     M|
+------+



In [11]:
# TODO: write your code to answer question 3
user_log.groupby(user_log['gender']).agg(countDistinct("userId")).show()

+------+----------------------+
|gender|count(DISTINCT userId)|
+------+----------------------+
|     F|                   462|
|  null|                     1|
|     M|                   501|
+------+----------------------+



In [36]:
user_log.where(user_log.gender=='F').select('userId','gender').agg(countDistinct("userId")).show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [59]:
# TODO: write your code to answer question 4
user_log.where(user_log.artist==user_log.artist).groupby("artist").count().sort(desc("count")).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
|Red Hot Chili Pep...|   24|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [66]:
# TODO: write your code to answer question 5
# filter out 0 sum and max sum to get more exact anser
function=udf(lambda ishome: int(ishome=='Home'), IntegerType())

user_window=Window.partitionBy('userId').orderBy(desc('ts'))

cusum=user_log.where((user_log.page=='NextSong') | (user_log.page=='Home'))\
        .select('userId','page','ts') \
        .withColumn('homevisit',function(user_log.page))\
        .withColumn('period', Fsum('homevisit').over(user_window))
cusum.show()

+------+--------+-------------+---------+------+
|userId|    page|           ts|homevisit|period|
+------+--------+-------------+---------+------+
|  1436|NextSong|1513783259284|        0|     0|
|  1436|NextSong|1513782858284|        0|     0|
|  2088|    Home|1513805972284|        1|     1|
|  2088|NextSong|1513805859284|        0|     1|
|  2088|NextSong|1513805494284|        0|     1|
|  2088|NextSong|1513805065284|        0|     1|
|  2088|NextSong|1513804786284|        0|     1|
|  2088|NextSong|1513804555284|        0|     1|
|  2088|NextSong|1513804196284|        0|     1|
|  2088|NextSong|1513803967284|        0|     1|
|  2088|NextSong|1513803820284|        0|     1|
|  2088|NextSong|1513803651284|        0|     1|
|  2088|NextSong|1513803413284|        0|     1|
|  2088|NextSong|1513803254284|        0|     1|
|  2088|NextSong|1513803057284|        0|     1|
|  2088|NextSong|1513802824284|        0|     1|
|  2162|NextSong|1513781246284|        0|     0|
|  2162|NextSong|151

In [107]:
result=cusum.where(cusum.page=='NextSong')\
    .groupBy('userId','period')\
    .agg({'period':'count'})\
    .agg({'count(period)':'avg'})
result.show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [109]:
result.withColumn('avgsong',round(result['avg(count(period))']).cast('integer')).show()

+------------------+-------+
|avg(count(period))|avgsong|
+------------------+-------+
| 6.898347107438017|      7|
+------------------+-------+



In [87]:
# wrong answer
#user_log.where(user_log.song==user_log.song).groupBy('userId').count()\
#    .agg(avg('count')).show()

+----------------+
|      avg(count)|
+----------------+
|9.87810650887574|
+----------------+

